In [15]:
import pandas as pd
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyOAuth
import networkx as nx
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objects as go
import json
import plotly.express as px
import kaleido
import plotly.io as pio
import os
from pyquery import PyQuery as pq

pio.kaleido.scope.mathjax = None

layout_dict = {
    'plot_bgcolor': 'white',
    'paper_bgcolor': 'white',
    'titlefont': {
        'size': 36
    },
    'font': {
        'family': "SansSerif",
        'color': "black",
        'size': 24
    },
    'yaxis': {
        'showgrid': False,
        'zeroline': False,
        'showline': False,
        'showticklabels': True,
        'color': "black"
    },
    'xaxis': {
        'showgrid': False,
        'zeroline': False,
        'showline': False,
        'color': "black",
        'showticklabels': True
    }
}

In [20]:
twitter = pd.read_csv("Clean_Data/Twitter.csv")
facebook = pd.read_csv("Clean_Data/Facebook.csv")
facebook = facebook[facebook.year > 2013]
facebook = facebook[facebook.year < 2021]
google = pd.read_csv("Clean_Data/Google.csv")
apple = pd.read_csv("Clean_Data/Apple.csv")
microsoft = pd.read_csv("Clean_Data/Microsoft.csv")
zoom = pd.read_csv("Clean_Data/Zoom.csv")

twitter["site"] = "Twitter"
facebook["site"] = "Facebook"
google["site"] = "Google"
apple["site"] = "Apple"
microsoft["site"] = "Microsoft"

google['accounts_per_warrant'] = google.apply(lambda t: t.warrant_accounts / t.warrants, axis=1)
google['accounts_per_subpoena'] = google.apply(lambda t: t.subpoena_accounts / t.subpoenas, axis=1)
google["time"] = google.apply(lambda y: ("Jan-Jun" if y.half == 1 else "Jul-Dec") + " " + str(y.year), axis=1)

facebook.search_warrant = facebook.search_warrant.map(lambda t: int(t.replace(',', '')))
facebook.search_warrant_accounts = facebook.search_warrant_accounts.map(lambda t: int(t.replace(',', '')))
facebook.subpoena_accounts = facebook.subpoena_accounts.map(lambda t: int(t.replace(',', '')))
facebook.subpoena = facebook.subpoena.map(lambda t: int(t.replace(',', '')))

facebook['accounts_per_warrant'] = facebook.apply(lambda t: t.search_warrant_accounts / t.search_warrant, axis=1)
facebook['accounts_per_subpoena'] = facebook.apply(lambda t: t.subpoena_accounts / t.subpoena, axis=1)
facebook["time"] = facebook.apply(lambda y: ("Jan-Jun" if y.half == 1 else "Jul-Dec") + " " + str(y.year), axis=1)

common_names = ["site", "year", "half", "total_requests", "total_requests_accounts"]

df = pd.concat([
    twitter[common_names], 
    facebook[common_names], 
    google[common_names], 
    apple[common_names], 
    microsoft[common_names]
])
df = df[df.year > 2013]
df = df.dropna()

In [27]:
facebook.head()

,Unnamed: 0,country,total_requests,total_requests_accounts,search_warrant,search_warrant_accounts,subpoena,subpoena_accounts,nsls,nsls_accounts,year,half,site,accounts_per_warrant,accounts_per_subpoena,time
2,3,United States,15433.0,23667.0,7669,12222,6087,9488,NaN,NaN,2014,1,Facebook,1.593689,1.558732,Jan-Jun 2014
3,4,United States,14274.0,21731.0,7924,12403,4638,7278,NaN,NaN,2014,2,Facebook,1.565245,1.569211,Jul-Dec 2014
4,5,United States,17577.0,26579.0,9737,15013,5375,8475,NaN,NaN,2015,1,Facebook,1.541851,1.576744,Jan-Jun 2015
5,6,United States,19235.0,30041.0,11128,17703,5288,8134,NaN,NaN,2015,2,Facebook,1.590852,1.538200,Jul-Dec 2015
6,7,United States,23854.0,38951.0,13742,21626,6526,12447,NaN,NaN,2016,1,Facebook,1.573716,1.907294,Jan-Jun 2016


In [38]:
accounts_slides = go.Figure(data=[
    go.Scatter(x=google.time, 
           y=google.warrants,
           marker=dict(color="red",size=18),
           name="Google",
           mode="markers"),
    go.Scatter(x=facebook.time,
           y=facebook.search_warrant,
           marker=dict(color="blue",size=18,symbol="triangle-up"),
           name="Facebook",
           mode="markers")
])

accounts_slides.update_layout(layout_dict)
accounts_slides.update_layout({    
    'title': "Warrants Issued to Facebook and Google",
    'xaxis_title':"Year",
    'yaxis_title': "Warrants"
})

accounts_slides.write_html("Graphics/Slides/warrants.html")
accounts_slides.show()

In [39]:
accounts_slides = go.Figure(data=[
    go.Scatter(x=google.time, 
           y=google.accounts_per_warrant, 
           marker=dict(color = "red",size=18),
          name="Google",
              mode="markers"),
    go.Scatter(x=facebook.time,
          y=facebook.accounts_per_warrant,
          marker=dict(color="blue",size=18,symbol="triangle-up"),
          name="Facebook",
              mode="markers")
])

accounts_slides.update_layout(layout_dict)
accounts_slides.update_layout({    
    'title': "Accounts Per Warrant",
    'xaxis_title':"Year",
    'yaxis_title': "Accounts Per Warrant"
})

accounts_slides.write_html("Graphics/Slides/accounts_per_warrant.html")
accounts_slides.show()

In [7]:
tmp_df = google.groupby("year").sum()

accounts_slides = go.Figure(data=[
    go.Bar(name="Accounts per Warrant", 
           x=tmp_df.index, 
           y=tmp_df.accounts_per_warrant,
          marker=dict(color = "red")),
    go.Bar(name="Accounts per Subpoena", 
           x=tmp_df.index,
           y=tmp_df.accounts_per_subpoena,
          marker=dict(color = "salmon"))
])

accounts_slides.update_layout(layout_dict)
accounts_slides.update_layout({    
    'title': "Accounts per Warrant and Subpoena Issued to Google",
    'xaxis_title':"Year",
    'yaxis_title': "Accounts Per Request"
})

accounts_slides.write_html("Graphics/Slides/warrant_and_subpoena_issued_to_google.html")
accounts_slides.show()

In [8]:
tmp_df = facebook.groupby("year").sum()

accounts_slides = go.Figure(data=[
    go.Bar(x=tmp_df.index, y=tmp_df.search_warrant),
])

accounts_slides.update_layout(layout_dict)
accounts_slides.update_layout({    
    'title': "Warrants Issued to Facebook",
    'xaxis_title':"Year",
    'yaxis_title': "Warrants"
})

accounts_slides.write_html("Graphics/warrants_issued_to_facebook.html")
accounts_slides.show()

In [85]:
tmp_df = facebook.groupby("year").sum()

accounts_slides = go.Figure(data=[
    go.Bar(x=tmp_df.index, y=tmp_df.accounts_per_warrant),
])

accounts_slides.update_layout(layout_dict)
accounts_slides.update_layout({    
    'title': "User Information Warrants Issued to Facebook",
    'xaxis_title':"Year",
    'yaxis_title': "Accounts Per Warrant"
})

accounts_slides.write_html("Graphics/accounts_per_warrant_slides.html")
accounts_slides.show()